In [1]:
import pandas as pd
import numpy as np
import sys
import time

sys.path.append('../../../')
from utils import modelling as mod
from utils import utils
from utils import paths
import wandb

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import KFold, cross_val_score, cross_validate

In [2]:
# df = pd.read_csv('/Users/tudor/Documents/phd/coding/project_CF/data/final_df_aberdeen.csv')
df = pd.read_csv('/gws/nopw/j04/ai4er/users/ts809/era5_final/final_df_aberdeen.csv')    # JASMIN

weights = mod.CalcClassWeights(df['floods'])
df_to_analyse = df.drop(columns=['Unnamed: 0.1', 'time', 'Unnamed: 0', 'floods', 'time_ok', 't'])

X = df_to_analyse.drop(columns=['floods_x4'])
y = df_to_analyse['floods_x4']

X_train = X.iloc[0:int(len(X) *.8)] 
X_test = X.iloc[int(len(X) *.8):int(len(X) *.9)]
X_val = X.iloc[int(len(X) *.9):]

y_train = y.iloc[0:int(len(y) *.8)]
y_test = y.iloc[int(len(y) *.8):int(len(y) *.9)]
y_val = y.iloc[int(len(y) *.9):]

In [3]:
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)
X_val = scaler.transform(X_val)

In [4]:
dectree = DecisionTreeClassifier(
    criterion = 'gini',
    splitter='best',
    max_depth=None,
    min_samples_split = 50,
    min_samples_leaf= 1,
    max_features='sqrt',
    random_state=42,
    class_weight=weights,
)

	# criterion: gini
	# max_depth: None
	# max_features: sqrt
	# min_samples_leaf: 1
	# min_samples_split: 50
	# scaler: minmax
	# splitter: best

In [5]:
cv = KFold(n_splits = 4, shuffle=False)

# start = time.time()

cv_scores = cross_validate(dectree, X_train, y_train, cv=cv, 
                        scoring=('balanced_accuracy', 'jaccard', 'roc_auc_ovr', 'neg_log_loss'),
                        n_jobs=4,
                        error_score='raise',
)

In [6]:
print(cv_scores.keys())
interm = time.time()
BA_score = cv_scores['test_balanced_accuracy'].mean()
JAC_score = cv_scores['test_jaccard'].mean()
AUC_score = cv_scores['test_roc_auc_ovr'].mean()
CE_score = cv_scores['test_neg_log_loss'].mean()

BA_score

dict_keys(['fit_time', 'score_time', 'test_balanced_accuracy', 'test_jaccard', 'test_roc_auc_ovr', 'test_neg_log_loss'])


np.float64(0.5053022143248644)

In [17]:
y_pred = dectree.predict(X_test)

In [18]:
from sklearn.metrics import balanced_accuracy_score
balanced_accuracy_score(y_test, y_pred)

0.4999571183533448